### save the metrics (static ones and lss ones) for v1.3 dbs.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import os
import pandas as pd
%matplotlib inline

In [2]:
data_dir = '/global/cscratch1/sd/awan/lsst_output/post_wp_output/summary_data/'
outdir = '/global/homes/a/awan/LSST/lsstRepos/ObsStrat/postwp/paper-data/summary_csv/'
os.makedirs(outdir, exist_ok=True)

### read in the data for the static metrics and store as a dictionary

In [3]:
# read in the data 
data = {}
yr_label = {}
for yr in [1, 3, 6, 10]:
    files = [f for f in os.listdir( data_dir ) if f.endswith('csv') and f.__contains__('y%s_' % yr) and f.startswith('eg_') ]
    for file in files:
        print( file )
        key = 'yr%s_%s' % (yr, file.split('_')[4])
        data[ key ] = pd.read_csv('%s/%s' % (data_dir, file))
        yr_label[ key ] =  r'Y%s (i$>$%s) ' % (yr, file.split('_')[4].split('limi')[-1])
        
print( yr_label )
print(data.keys())
print(data[list(data.keys())[0]].keys() )

eg_footprint_stats_y1_limi24.75_nodither_nside256.csv
eg_footprint_stats_y3_limi25.35_nodither_nside256.csv
eg_footprint_stats_y6_limi25.72_nodither_nside256.csv
eg_footprint_stats_y10_limi26.0_nodither_nside256.csv
{'yr1_limi24.75': 'Y1 (i$>$24.75) ', 'yr3_limi25.35': 'Y3 (i$>$25.35) ', 'yr6_limi25.72': 'Y6 (i$>$25.72) ', 'yr10_limi26.0': 'Y10 (i$>$26.0) '}
dict_keys(['yr1_limi24.75', 'yr3_limi25.35', 'yr6_limi25.72', 'yr10_limi26.0'])
Index(['dbname', 'Area (deg2)', '$i$-band depth: median',
       ' $i$-band depth: std'],
      dtype='object')


In [4]:
# read in the static metrics
for key in data:
    yr_tag = key.split('_')[0].split('yr')[-1]
    lim_tag = key.split('_')[1].split('limi')[-1]
    print(yr_tag, lim_tag)

    data[key] = data[key].rename(columns={'Area (deg2)': 'Y%s effective survey area ' % (yr_tag),
                                          '$i$-band depth: median': 'median Y%s $i$-band coadded depth in effective survey area' % (yr_tag),
                                          ' $i$-band depth: std': 'std in Y%s $i$-band coadded depth in effective survey area' % (yr_tag)} )
    #fname = 'lss_metrics_%s.csv' % key
    #data[key].to_csv('%s/%s' % (outdir, fname), index=False)

1 24.75
3 25.35
6 25.72
10 26.0


In [5]:
# just check things
data['yr1_limi24.75']

,dbname,Y1 effective survey area,median Y1 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area
0,templates_w_2.0_1exp_pairsmix_10yrs,15188.75,25.19,0.14
1,shortt_5ns_5ext_pairsmix_10yrs,15140.12,25.20,0.15
2,roll_mod6_sdf0.20mixed_10yrs,15258.78,25.20,0.14
3,weather_0.60c_10yrs,15356.35,25.27,0.15
4,baseline10yrs,15258.78,25.20,0.14
...,...,...,...,...
170,dcr_nham2_v1.3_10yrs,14939.95,25.12,0.15
171,templatew5.00_v1.3_10yrs,15160.27,25.20,0.15
172,filterload_illum30_v1.3_10yrs,15088.14,25.20,0.15
173,tde_illum75_v1.3_10yrs,15266.80,25.17,0.14


#### read in the given `opsim_runs` file: gives us the order of the 

In [6]:
given_list = pd.read_csv('paper-data/opsim_runs.csv')
given_list = given_list.drop('Unnamed: 0', axis=1) # drop the index column
given_list

,name,group
0,agnddf_illum10_v1.3_10yrs.db,AGN_DDF
1,agnddf_illum15_v1.3_10yrs.db,AGN_DDF
2,agnddf_illum30_v1.3_10yrs.db,AGN_DDF
3,agnddf_illum60_v1.3_10yrs.db,AGN_DDF
4,dcr_nham1_v1.3_10yrs.db,DCR
...,...,...
89,wfd_95_v1.3_10yrs.db,wfd_depth
90,wfd_standard_v1.3_10yrs.db,wfd_depth
91,wfd_only_2snap_v1.3_10yrs.db,wfd_only
92,wfd_only_nomix_v1.3_10yrs.db,wfd_only


In [7]:
# clean up some things:
# drop baseline_v1.3_1yrs.db
ind_drop = np.where( given_list['name'].values == 'baseline_v1.3_1yrs.db' )[0]
print(ind_drop)

# drop repeated entries
more = np.where( given_list['name'].values == 'baseline_v1.3_10yrs.db' )[0]
print(more)
ind_drop = np.hstack( [ind_drop, more[1:]] )
print(ind_drop)

given_list = given_list.drop(ind_drop)
modified_list = given_list.reset_index(drop=True)
given_list = []
given_list = modified_list
given_list

[19]
[20 37 47]
[19 37 47]


,name,group
0,agnddf_illum10_v1.3_10yrs.db,AGN_DDF
1,agnddf_illum15_v1.3_10yrs.db,AGN_DDF
2,agnddf_illum30_v1.3_10yrs.db,AGN_DDF
3,agnddf_illum60_v1.3_10yrs.db,AGN_DDF
4,dcr_nham1_v1.3_10yrs.db,DCR
...,...,...
86,wfd_95_v1.3_10yrs.db,wfd_depth
87,wfd_standard_v1.3_10yrs.db,wfd_depth
88,wfd_only_2snap_v1.3_10yrs.db,wfd_only
89,wfd_only_nomix_v1.3_10yrs.db,wfd_only


In [8]:
# check some more things:
len( list( given_list['name'] ) ), len( np.unique( given_list['name'] ) )

(91, 91)

#### find the indices to get the order

In [9]:
# find the indices to get the order
ind = []
for db in modified_list['name']:
    ind_temp = np.where( data['yr1_limi24.75']['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

[112  95  89 162 121 170 152  91 128 150 137 113 165 124 119 122  92  93
 142 159 151 103 139  97 158 156 114 101 160 116 154 110 164 172 143 141
  84 144 147 123 125 135  87 117 153 131 132  86  85 115 140 168 133  94
 130 148 126 161  98  96 167 173 174 106 120 146 163 108 100 129 102 109
 171 127 166 118 105 138  88 136 149 111  90 157 155 169 107 134  99 104
 145]


In [10]:
# check we have the right indices
print( data['yr1_limi24.75']['dbname'].values[112],  given_list['name'][0] )

# check we have the right number of inds
print( len(ind), len(np.unique(ind)) )

agnddf_illum10_v1.3_10yrs agnddf_illum10_v1.3_10yrs.db
91 91


#### save the data in the order in which to save things

In [11]:
for yr_lim_tag in data:
    for colname in data[yr_lim_tag]:
        if colname != 'dbname':
             given_list[colname] = data[yr_lim_tag][colname].values[ind]

In [12]:
given_list

,name,group,Y1 effective survey area,median Y1 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,Y3 effective survey area,median Y3 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,Y6 effective survey area,median Y6 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,Y10 effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area
0,agnddf_illum10_v1.3_10yrs.db,AGN_DDF,15278.76,25.23,0.15,15305.78,25.80,0.14,15257.73,26.15,0.13,15280.29,26.44,0.13
1,agnddf_illum15_v1.3_10yrs.db,AGN_DDF,15253.27,25.22,0.15,15273.94,25.80,0.14,15248.34,26.15,0.13,15281.49,26.44,0.13
2,agnddf_illum30_v1.3_10yrs.db,AGN_DDF,15214.51,25.21,0.15,15234.60,25.79,0.14,15221.64,26.14,0.13,15246.29,26.43,0.13
3,agnddf_illum60_v1.3_10yrs.db,AGN_DDF,15248.86,25.23,0.15,15269.32,25.79,0.14,15226.15,26.14,0.13,15254.74,26.43,0.13
4,dcr_nham1_v1.3_10yrs.db,DCR,15038.25,25.16,0.15,15138.55,25.75,0.13,15054.31,26.09,0.12,15086.36,26.37,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,wfd_95_v1.3_10yrs.db,wfd_depth,15310.39,25.28,0.15,15312.86,25.86,0.14,15271.79,26.21,0.13,15291.35,26.50,0.13
87,wfd_standard_v1.3_10yrs.db,wfd_depth,15325.40,25.26,0.16,15356.87,25.83,0.14,15327.65,26.18,0.13,15356.56,26.47,0.13
88,wfd_only_2snap_v1.3_10yrs.db,wfd_only,15217.13,25.29,0.16,15253.58,25.87,0.14,15233.28,26.21,0.14,15259.83,26.50,0.13
89,wfd_only_nomix_v1.3_10yrs.db,wfd_only,15245.61,25.31,0.17,15290.93,25.89,0.15,15255.37,26.23,0.15,15285.06,26.53,0.14


In [13]:
# save the metrics
fname = 'static_metrics.csv'
given_list.to_csv('%s/%s' % (outdir, fname), index=False)

#### now read and save lss-fom metric

In [14]:
for key in given_list.keys():
    print(key)
    if key.__contains__('Y'):
        given_list = given_list.drop(key, axis=1)

name
group
Y1 effective survey area 
median Y1 $i$-band coadded depth in effective survey area
std in Y1 $i$-band coadded depth in effective survey area
Y3 effective survey area 
median Y3 $i$-band coadded depth in effective survey area
std in Y3 $i$-band coadded depth in effective survey area
Y6 effective survey area 
median Y6 $i$-band coadded depth in effective survey area
std in Y6 $i$-band coadded depth in effective survey area
Y10 effective survey area 
median Y10 $i$-band coadded depth in effective survey area
std in Y10 $i$-band coadded depth in effective survey area


In [15]:
data_file = '/global/cscratch1/sd/awan/lsst_output/post_wp_output/lss_fom/os_bias_overplots/'
data_file += '2019-12-18_fomdata_Y10_i<25.3_snfom_egfootprint_91cadences_0.66<z<1.0_th-r<25.70_100<ell<300_logtcuts.csv'
lss_fom = pd.read_csv(data_file)

In [16]:
lss_fom

,dbname,dither,fsky_best,fsky,FoM,meta
0,newB_v1.3_10yrs,NoDither,0.368823,0.398343,1.019117,ugri_noPoisson_no0pt
1,twilight_neo_mod3_v1.3_10yrs,NoDither,0.368823,0.366879,0.978531,ugri_noPoisson_no0pt
2,agnddf_illum30_v1.3_10yrs,NoDither,0.368823,0.369581,0.983555,ugri_noPoisson_no0pt
3,wfd_75_v1.3_10yrs,NoDither,0.368823,0.374193,0.976530,ugri_noPoisson_no0pt
4,descddf_illum3_v1.3_10yrs,NoDither,0.368823,0.371774,0.938117,ugri_noPoisson_no0pt
...,...,...,...,...,...,...
86,delayedrolling_mod2_sdf_0.10_v1.3_10yrs,NoDither,0.368823,0.365351,0.976252,ugri_noPoisson_no0pt
87,wfd_90_v1.3_10yrs,NoDither,0.368823,0.371880,0.984409,ugri_noPoisson_no0pt
88,dcr_nham2_v1.3_10yrs,NoDither,0.368823,0.363889,0.976525,ugri_noPoisson_no0pt
89,templatew5.00_v1.3_10yrs,NoDither,0.368823,0.368880,0.981225,ugri_noPoisson_no0pt


In [17]:
# find the indices to get the order
ind = []
for db in modified_list['name']:
    ind_temp = np.where( lss_fom['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

[27  8  2 79 36 88 68  4 43 66 52 28 82 39 34 37  5  6 57 76 67 16 54 10
 75 72 29 14 77 31 70 25 81 90 58 56 74 59 62 38 40 50  0 32 69 46 47 83
 19 30 55 86 48  7 45 64 41 78 11  9 85 23 63 20 35 61 80 22 13 44 15 24
 89 42 84 33 18 53  1 51 65 26  3 73 71 87 21 49 12 17 60]


In [18]:
given_list['lss fom'] = lss_fom['FoM'].values[ind]

In [19]:
fname = 'lss_metric.csv'
given_list.to_csv('%s/%s' % (outdir, fname), index=False)